In [87]:
import math
import numpy as np
from collections import Counter
from functools import partial, reduce

# https://www.saedsayad.com/decision_tree.htm

In [89]:
# Outlook, Temperature, Humidity, Windy, Play Golf.
OUTLOOK = 0
TEMPERATURE = 1
HUMIDITY = 2
WINDY = 3
PLAY_GOLF = -1

data = """rainy hot high false no
rainy hot high true no
overcast hot high false yes
sunny mild high false yes
sunny cool normal false yes
sunny cool normal true no
overcast cool normal true yes
rainy mild high false no
rainy cool normal false yes
sunny mild normal false yes
rainy mild normal true yes
overcast mild high true yes
overcast hot normal false yes
sunny mild high true no"""

data = np.array(list((map(lambda s: s.split(' '), data.split('\n')))))
data[:2]

array([['rainy', 'hot', 'high', 'false', 'no'],
       ['rainy', 'hot', 'high', 'true', 'no']], dtype='<U8')

In [62]:
math.log(5) / math.log(2) == math.log2(5)

True

In [90]:
## Entropy

def entropy(classes):
    # The entropy is completely homogenous (there are no split).
    if len(classes) < 2: return 0
    [t, f] = classes
    """Calculates the entropy given the number of true and false instances."""
    
    if f == 0: return 0
    n = t + f
    fn = lambda p: -p * math.log2(p) 
    return fn(t / n) + fn(f / n)

In [132]:
# Example using the frequency table of one attribute.
# Partition by the last column "Play Golf" with the value "yes" and "no".
# yes, no = partition_by(data, -1, 'yes')
# len(yes), len(no)
labels = Counter(data[:,-1])
labels.items()

dict_items([('no', 5), ('yes', 9)])

In [133]:
# To find the entropy of the target, E(PlayGold) = E(9, 5).
entropy_target = entropy(labels.values()) # entropy(9, 5), # 0.9402859586706311
entropy_target

0.9402859586706311

In [128]:
def entropy_for(data, col):
    total = 0.0
    n = len(data)
    
    # Get the unique values for the given columns.
    attributes = set(data[:, col])
    
    # For each values, find the number of rows that matches the attribute
    # in order to count the target classes.
    for attribute in attributes:
        rows = data[data[:, col] == attribute]
        labels = Counter(rows[:, -1])
        print(f'{attribute:10s}: {labels}')
        total += entropy(labels.values()) * len(rows) / n
    return total

In [129]:
entropy_for(data, OUTLOOK) # 0.6935361388961919

sunny     : Counter({'yes': 3, 'no': 2})
overcast  : Counter({'yes': 4})
rainy     : Counter({'no': 3, 'yes': 2})


0.6935361388961919

In [130]:
entropy_for(data, HUMIDITY) # 0.7884504573082896

high      : Counter({'no': 4, 'yes': 3})
normal    : Counter({'yes': 6, 'no': 1})


0.7884504573082896

In [131]:
entropy_for(data, TEMPERATURE) # 0.9110633930116763

mild      : Counter({'yes': 4, 'no': 2})
cool      : Counter({'yes': 3, 'no': 1})
hot       : Counter({'no': 2, 'yes': 2})


0.9110633930116764

In [122]:
entropy_for(data, WINDY) # 0.8921589282623617

false     : Counter({'yes': 6, 'no': 2})
true      : Counter({'no': 3, 'yes': 3})


0.8921589282623617

In [123]:
entropy_for(data, PLAY_GOLF) # The target entropy has to be calculated separately.

no        : Counter({'no': 5})
yes       : Counter({'yes': 9})


0.0

## Information Gain

The information gained is based on the decrease in entropy after a dataset is split on an attribute. The branch is divided based on the attribute that returns the highest information gain. This means the attribute with the higher information gain is preferred.

In [ ]:
# Gain(T, X) = Entropy(T) - Entropy(T, X)
# G(PlayGolf, Outlook) = E(PlayGolf) - E(PlayGolf, Outlook)
entropy_target - entropy_for(data, OUTLOOK) # 0.24674981977443922

In [ ]:
# Information Gain.
# (('outlook', 0.24674981977443922),
#  ('humidity', 0.15183550136234159),
#  ('temperature', 0.02922256565895487),
#  ('windy', 0.04812703040826949))
labels = ['outlook', 'humidity', 'temperature', 'windy']
for i in range(4):
    print(labels[i],':', entropy_target - entropy_for(data, i))
    print()

## Gini Index

Gini index is a metric to measure how often a randomly chosen element would be incorrectly identified. Attributes with a lower gini index is preferred.

In [ ]:
def gini(t, f=0):
    if f == 0: return 0
    fn = lambda p: p ** 2
    n = t + f
    return 1 - fn(t / n) - fn(f / n)

In [ ]:
gini(5, 7)

In [ ]:
def gini_for(data, col):
    total = 0.0
    n = len(data)
    attributes = counter(data, col)
    for attribute, _ in attributes:
        o, _ = partition_by(data, col, attribute)
        print(f'{attribute:10s}: {counter(o)}')
        total += gini(*map(pick_value, counter(o))) * len(o) / n
    return total

In [70]:
for i in range(4):
    print(gini_for(data, i))
    print()

NameError: name 'gini_for' is not defined